In [1]:
import scipy.sparse as sp
import numpy as np

In [2]:
featureMatrix = sp.load_npz("feature_matrix.npz")

In [3]:
featureMatrix

<19579x24951 sparse matrix of type '<class 'numpy.int64'>'
	with 250731 stored elements in Compressed Sparse Row format>

In [4]:
featureMatrix.shape

(19579, 24951)

In [5]:
# Testing if generated matrix was okay or not
np.where(featureMatrix.toarray()[0] == 1)

(array([  453,  1286,  1636,  3702,  6102,  6862,  8189, 10730, 13337,
        13640, 13875, 16003, 16564, 17069, 18513, 19478, 19625, 23225,
        24168, 24416, 24641]),)

In [6]:
# Looks good to me